# Data Acquisition: Cancer Digital Slide Archive

This notebook illustrates the steps used to retrieve data from http://cancer.digitalslidearchive.net/. This process was mostly automated, and primarily relied on web scraping methods.

In [21]:
from bs4 import BeautifulSoup as BS
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import urllib.request
import re
import joblib
num_cores = 8

Initialize empty dataframe:

In [3]:
pdf = pd.DataFrame(columns = ['url', 'collection'])

Read text files from directory. These text files were created by manually browsing each collection and copying the data into the respective .txt files.

In [4]:
mypath = '/Users/aadi/Google Drive/School/MS Data Analytics/Master\'s Project/tcga_scraping'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [5]:
onlyfiles

['acc.txt',
 'blca.txt',
 'brca.txt',
 'cesc.txt',
 'chol.txt',
 'coad.txt',
 'dlbc.txt',
 'esca.txt',
 'gbm.txt',
 'hnsc.txt',
 'kich.txt',
 'kirp.txt',
 'lgg.txt',
 'lihc.txt',
 'luad.txt',
 'lusc.txt',
 'meso.txt',
 'ov.txt',
 'paad.txt',
 'pcpg.txt',
 'prad.txt',
 'read.txt',
 'sarc.txt',
 'skcm.txt',
 'stad.txt',
 'tgct.txt',
 'thca.txt',
 'thym.txt',
 'ucec.txt',
 'ucs.txt',
 'uvm.txt']

Retrieve img tags from the html documents. These tags correspond to thumbnails displayed for each slide:

In [8]:
for file in onlyfiles:
    print(file)
    with open(mypath+'/'+file, 'r') as myfile:
        data=myfile.read().replace('\n', '')

    soup = BS(data, "lxml")

    for imgtag in soup.find_all('img'):
        pdf = pdf.append({"url": imgtag, "collection":file}, ignore_index=True)
pdf_bk = pdf

acc.txt
blca.txt
brca.txt
cesc.txt
chol.txt
coad.txt
dlbc.txt
esca.txt
gbm.txt
hnsc.txt
kich.txt
kirp.txt
lgg.txt
lihc.txt
luad.txt
lusc.txt
meso.txt
ov.txt
paad.txt
pcpg.txt
prad.txt
read.txt
sarc.txt
skcm.txt
stad.txt
tgct.txt
thca.txt
thym.txt
ucec.txt
ucs.txt
uvm.txt


Data Cleaning: Convert BS4 objects to strings, and change thumbnail width from 200 to 500:

In [9]:
pdf['collection'] = pdf['collection'].astype(str)
pdf['collection'] = pdf['collection'].str.replace('.txt', '')

In [10]:
pdf['url'] = pdf['url'].astype(str)
pdf = pdf[pdf['url'].str.contains("emory")]
pdf['url'] = pdf['url'].str.replace('WID=200', 'WID=500')

,url,collection
0,"<img src=""local_images/CDSA_Slide_50.png"" styl...",acc
1,"<img height=""85px"" src=""imgs/Winship_06-2011/W...",acc
2,"<img class=""dhx_combo_img"" src=""dsa-common-fil...",acc
3,"<img src=""http://node15.cci.emory.edu/cgi-bin/...",acc
4,"<img src=""http://node15.cci.emory.edu/cgi-bin/...",acc
5,"<img src=""http://node15.cci.emory.edu/cgi-bin/...",acc
6,"<img src=""http://node15.cci.emory.edu/cgi-bin/...",acc
7,"<img src=""http://node15.cci.emory.edu/cgi-bin/...",acc
8,"<img src=""http://node15.cci.emory.edu/cgi-bin/...",acc
9,"<img src=""http://node15.cci.emory.edu/cgi-bin/...",acc


Gather urls from between quotation marks, and construct slide names:

In [13]:
pdf['url'] = pdf['url'].str.extract('"([^"]*)"')
pdf_bk = pdf
pdf['url'] = pdf['url'].str.replace('&amp;', '&')
pdf['name'] = 'TCGA'+pdf['url'].str.extract('TCGA([^\.]*)\.')

/Users/aadi/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
/Users/aadi/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


'http://node15.cci.emory.edu/cgi-bin/iipsrv.fcgi?FIF=/bigdata2/PYRAMIDS/CDSA/BRCA_Frozen/nationwidechildrens.org_BRCA.tissue_images.Level_1.147.1.0/TCGA-D8-A27N-01A-01-TS1.f4371b61-6f8f-48c6-9493-d9c96402634f.svs.dzi.tif&amp;WID=500&amp;CVT=jpeg'

#### Download Thumbnails:

In [22]:
folder_names = pdf['collection'].unique()
img_path = '/Users/aadi/Documents/tcga_imgs/thumbnails/'

def process_download(url_idx, df_slice, category):
    img_filename = img_path + category + '/' + df_slice['name'].iloc[url_idx] + ".jpg"
    img_url = df_slice.iloc[url_idx, 0]
    try:
        urllib.request.urlretrieve(img_url, img_filename)
    except:
        pass
    print(category+': '+str(url_idx+1)+'/'+str(len(df_slice['url'])+1))
    
for category in folder_names[13:]:
    category_dir = img_path+category
    if not os.path.exists(category_dir):
        os.makedirs(category_dir)
    df_slice = pdf[pdf['collection'].str.contains(category)]
    joblib.Parallel(n_jobs=num_cores)(joblib.delayed(process_download)(i, df_slice, category) for i, image in enumerate(df_slice['url']))


lihc: 4/1028
lihc: 6/1028
lihc: 2/1028
lihc: 1/1028
lihc: 8/1028
lihc: 3/1028
lihc: 10/1028
lihc: 12/1028
lihc: 5/1028
lihc: 7/1028
lihc: 11/1028
lihc: 9/1028
lihc: 14/1028
lihc: 16/1028
lihc: 15/1028
lihc: 25/1028
lihc: 13/1028
lihc: 23/1028
lihc: 27/1028
lihc: 17/1028
lihc: 29/1028
lihc: 31/1028
lihc: 19/1028
lihc: 21/1028
lihc: 18/1028
lihc: 26/1028
lihc: 24/1028
lihc: 28/1028
lihc: 20/1028
lihc: 33/1028
lihc: 30/1028
lihc: 35/1028
lihc: 32/1028
lihc: 37/1028
lihc: 22/1028
lihc: 39/1028
lihc: 45/1028
lihc: 40/1028
lihc: 43/1028
lihc: 34/1028
lihc: 38/1028
lihc: 41/1028
lihc: 36/1028
lihc: 46/1028
lihc: 47/1028
lihc: 42/1028
lihc: 44/1028
lihc: 49/1028
lihc: 48/1028
lihc: 51/1028
lihc: 55/1028
lihc: 53/1028
lihc: 50/1028
lihc: 57/1028
lihc: 61/1028
lihc: 52/1028
lihc: 59/1028
lihc: 63/1028
lihc: 65/1028
lihc: 54/1028
lihc: 56/1028
lihc: 67/1028
lihc: 66/1028
lihc: 62/1028
lihc: 58/1028
lihc: 60/1028
lihc: 64/1028
lihc: 71/1028
lihc: 73/1028
lihc: 77/1028
lihc: 79/1028
lihc: 69/1028
l

lihc: 556/1028
lihc: 557/1028
lihc: 554/1028
lihc: 559/1028
lihc: 558/1028
lihc: 561/1028
lihc: 560/1028
lihc: 563/1028
lihc: 562/1028
lihc: 566/1028
lihc: 567/1028
lihc: 564/1028
lihc: 569/1028
lihc: 571/1028
lihc: 565/1028
lihc: 574/1028
lihc: 576/1028
lihc: 577/1028
lihc: 572/1028
lihc: 578/1028
lihc: 570/1028
lihc: 573/1028
lihc: 575/1028
lihc: 568/1028
lihc: 584/1028
lihc: 585/1028
lihc: 580/1028
lihc: 579/1028
lihc: 583/1028
lihc: 587/1028
lihc: 589/1028
lihc: 591/1028
lihc: 588/1028
lihc: 592/1028
lihc: 593/1028
lihc: 594/1028
lihc: 590/1028
lihc: 582/1028
lihc: 599/1028
lihc: 586/1028
lihc: 596/1028
lihc: 595/1028
lihc: 581/1028
lihc: 601/1028
lihc: 600/1028
lihc: 603/1028
lihc: 602/1028
lihc: 605/1028
lihc: 604/1028
lihc: 607/1028
lihc: 609/1028
lihc: 606/1028
lihc: 608/1028
lihc: 611/1028
lihc: 598/1028
lihc: 610/1028
lihc: 617/1028
lihc: 612/1028
lihc: 614/1028
lihc: 615/1028
lihc: 619/1028
lihc: 618/1028
lihc: 621/1028
lihc: 620/1028
lihc: 613/1028
lihc: 623/1028
lihc: 616/

luad: 80/1447
luad: 78/1447
luad: 82/1447
luad: 86/1447
luad: 81/1447
luad: 83/1447
luad: 84/1447
luad: 85/1447
luad: 88/1447
luad: 87/1447
luad: 92/1447
luad: 90/1447
luad: 89/1447
luad: 91/1447
luad: 98/1447
luad: 93/1447
luad: 94/1447
luad: 95/1447
luad: 96/1447
luad: 97/1447
luad: 100/1447
luad: 99/1447
luad: 102/1447
luad: 101/1447
luad: 103/1447
luad: 107/1447
luad: 106/1447
luad: 105/1447
luad: 104/1447
luad: 108/1447
luad: 114/1447
luad: 111/1447
luad: 112/1447
luad: 113/1447
luad: 109/1447
luad: 110/1447
luad: 115/1447
luad: 119/1447
luad: 116/1447
luad: 123/1447
luad: 124/1447
luad: 120/1447
luad: 118/1447
luad: 122/1447
luad: 117/1447
luad: 126/1447
luad: 128/1447
luad: 125/1447
luad: 121/1447
luad: 130/1447
luad: 131/1447
luad: 132/1447
luad: 133/1447
luad: 129/1447
luad: 127/1447
luad: 136/1447
luad: 134/1447
luad: 135/1447
luad: 141/1447
luad: 137/1447
luad: 138/1447
luad: 143/1447
luad: 142/1447
luad: 139/1447
luad: 144/1447
luad: 140/1447
luad: 148/1447
luad: 147/1447
l

luad: 627/1447
luad: 629/1447
luad: 630/1447
luad: 628/1447
luad: 635/1447
luad: 631/1447
luad: 632/1447
luad: 633/1447
luad: 634/1447
luad: 636/1447
luad: 639/1447
luad: 638/1447
luad: 637/1447
luad: 640/1447
luad: 642/1447
luad: 643/1447
luad: 644/1447
luad: 641/1447
luad: 646/1447
luad: 645/1447
luad: 647/1447
luad: 648/1447
luad: 650/1447
luad: 651/1447
luad: 654/1447
luad: 652/1447
luad: 649/1447
luad: 655/1447
luad: 653/1447
luad: 658/1447
luad: 656/1447
luad: 660/1447
luad: 659/1447
luad: 662/1447
luad: 657/1447
luad: 663/1447
luad: 661/1447
luad: 664/1447
luad: 665/1447
luad: 666/1447
luad: 669/1447
luad: 670/1447
luad: 672/1447
luad: 667/1447
luad: 676/1447
luad: 671/1447
luad: 668/1447
luad: 673/1447
luad: 677/1447
luad: 675/1447
luad: 674/1447
luad: 679/1447
luad: 678/1447
luad: 680/1447
luad: 681/1447
luad: 684/1447
luad: 682/1447
luad: 683/1447
luad: 687/1447
luad: 688/1447
luad: 685/1447
luad: 689/1447
luad: 686/1447
luad: 690/1447
luad: 692/1447
luad: 691/1447
luad: 693/

luad: 1163/1447
luad: 1165/1447
luad: 1164/1447
luad: 1170/1447
luad: 1169/1447
luad: 1167/1447
luad: 1166/1447
luad: 1168/1447
luad: 1171/1447
luad: 1172/1447
luad: 1173/1447
luad: 1175/1447
luad: 1174/1447
luad: 1180/1447
luad: 1177/1447
luad: 1178/1447
luad: 1176/1447
luad: 1181/1447
luad: 1179/1447
luad: 1183/1447
luad: 1186/1447
luad: 1182/1447
luad: 1185/1447
luad: 1184/1447
luad: 1188/1447
luad: 1187/1447
luad: 1189/1447
luad: 1190/1447
luad: 1191/1447
luad: 1192/1447
luad: 1194/1447
luad: 1193/1447
luad: 1195/1447
luad: 1196/1447
luad: 1197/1447
luad: 1198/1447
luad: 1201/1447
luad: 1200/1447
luad: 1199/1447
luad: 1202/1447
luad: 1203/1447
luad: 1204/1447
luad: 1205/1447
luad: 1206/1447
luad: 1209/1447
luad: 1207/1447
luad: 1211/1447
luad: 1208/1447
luad: 1212/1447
luad: 1210/1447
luad: 1214/1447
luad: 1216/1447
luad: 1213/1447
luad: 1217/1447
luad: 1215/1447
luad: 1219/1447
luad: 1220/1447
luad: 1218/1447
luad: 1222/1447
luad: 1221/1447
luad: 1223/1447
luad: 1224/1447
luad: 12

...

thca: 739/1244
thca: 742/1244
thca: 738/1244
thca: 743/1244
thca: 741/1244
thca: 731/1244
thca: 745/1244
thca: 744/1244
thca: 748/1244
thca: 746/1244
thca: 751/1244
thca: 747/1244
thca: 740/1244
thca: 753/1244
thca: 750/1244
thca: 755/1244
thca: 752/1244
thca: 754/1244
thca: 756/1244
thca: 757/1244
thca: 759/1244
thca: 762/1244
thca: 639/1244
thca: 758/1244
thca: 761/1244
thca: 760/1244
thca: 769/1244
thca: 765/1244
thca: 763/1244
thca: 764/1244
thca: 766/1244
thca: 767/1244
thca: 771/1244
thca: 768/1244
thca: 770/1244
thca: 772/1244
thca: 775/1244
thca: 776/1244
thca: 774/1244
thca: 781/1244
thca: 778/1244
thca: 779/1244
thca: 777/1244
thca: 780/1244
thca: 782/1244
thca: 786/1244
thca: 783/1244
thca: 784/1244
thca: 788/1244
thca: 789/1244
thca: 785/1244
thca: 790/1244
thca: 787/1244
thca: 792/1244
thca: 791/1244
thca: 794/1244
thca: 796/1244
thca: 799/1244
thca: 798/1244
thca: 793/1244
thca: 795/1244
thca: 797/1244
thca: 803/1244
thca: 802/1244
thca: 800/1244
thca: 801/1244
thca: 804/

thym: 18/318
thym: 22/318
thym: 28/318
thym: 35/318
thym: 24/318
thym: 30/318
thym: 37/318
thym: 32/318
thym: 43/318
thym: 33/318
thym: 36/318
thym: 39/318
thym: 41/318
thym: 45/318
thym: 47/318
thym: 38/318
thym: 34/318
thym: 40/318
thym: 49/318
thym: 44/318
thym: 42/318
thym: 53/318
thym: 51/318
thym: 55/318
thym: 48/318
thym: 46/318
thym: 56/318
thym: 57/318
thym: 50/318
thym: 54/318
thym: 52/318
thym: 59/318
thym: 63/318
thym: 61/318
thym: 58/318
thym: 65/318
thym: 67/318
thym: 71/318
thym: 69/318
thym: 73/318
thym: 62/318
thym: 60/318
thym: 64/318
thym: 66/318
thym: 74/318
thym: 70/318
thym: 68/318
thym: 72/318
thym: 77/318
thym: 79/318
thym: 75/318
thym: 83/318
thym: 85/318
thym: 87/318
thym: 81/318
thym: 76/318
thym: 78/318
thym: 88/318
thym: 86/318
thym: 80/318
thym: 84/318
thym: 89/318
thym: 82/318
thym: 91/318
thym: 92/318
thym: 93/318
thym: 94/318
thym: 96/318
thym: 90/318
thym: 98/318
thym: 95/318
thym: 97/318
thym: 99/318
thym: 100/318
thym: 101/318
thym: 102/318
thym: 108

ucec: 291/1540
ucec: 290/1540
ucec: 286/1540
ucec: 294/1540
ucec: 295/1540
ucec: 298/1540
ucec: 292/1540
ucec: 293/1540
ucec: 296/1540
ucec: 302/1540
ucec: 299/1540
ucec: 301/1540
ucec: 297/1540
ucec: 304/1540
ucec: 300/1540
ucec: 307/1540
ucec: 305/1540
ucec: 308/1540
ucec: 303/1540
ucec: 313/1540
ucec: 311/1540
ucec: 306/1540
ucec: 310/1540
ucec: 315/1540
ucec: 317/1540
ucec: 312/1540
ucec: 309/1540
ucec: 316/1540
ucec: 319/1540
ucec: 318/1540
ucec: 314/1540
ucec: 323/1540
ucec: 320/1540
ucec: 322/1540
ucec: 326/1540
ucec: 324/1540
ucec: 321/1540
ucec: 327/1540
ucec: 325/1540
ucec: 328/1540
ucec: 333/1540
ucec: 329/1540
ucec: 334/1540
ucec: 330/1540
ucec: 332/1540
ucec: 331/1540
ucec: 339/1540
ucec: 336/1540
ucec: 338/1540
ucec: 335/1540
ucec: 337/1540
ucec: 340/1540
ucec: 346/1540
ucec: 341/1540
ucec: 342/1540
ucec: 344/1540
ucec: 244/1540
ucec: 349/1540
ucec: 343/1540
ucec: 348/1540
ucec: 347/1540
ucec: 353/1540
ucec: 345/1540
ucec: 355/1540
ucec: 351/1540
ucec: 359/1540
ucec: 352/

ucec: 837/1540
ucec: 840/1540
ucec: 844/1540
ucec: 836/1540
ucec: 842/1540
ucec: 841/1540
ucec: 843/1540
ucec: 839/1540
ucec: 838/1540
ucec: 847/1540
ucec: 834/1540
ucec: 848/1540
ucec: 846/1540
ucec: 849/1540
ucec: 854/1540
ucec: 851/1540
ucec: 852/1540
ucec: 850/1540
ucec: 857/1540
ucec: 853/1540
ucec: 856/1540
ucec: 861/1540
ucec: 862/1540
ucec: 859/1540
ucec: 863/1540
ucec: 855/1540
ucec: 865/1540
ucec: 858/1540
ucec: 867/1540
ucec: 864/1540
ucec: 866/1540
ucec: 870/1540
ucec: 860/1540
ucec: 868/1540
ucec: 875/1540
ucec: 871/1540
ucec: 869/1540
ucec: 874/1540
ucec: 872/1540
ucec: 876/1540
ucec: 879/1540
ucec: 873/1540
ucec: 877/1540
ucec: 880/1540
ucec: 878/1540
ucec: 881/1540
ucec: 884/1540
ucec: 883/1540
ucec: 882/1540
ucec: 885/1540
ucec: 890/1540
ucec: 886/1540
ucec: 889/1540
ucec: 845/1540
ucec: 891/1540
ucec: 887/1540
ucec: 895/1540
ucec: 888/1540
ucec: 897/1540
ucec: 893/1540
ucec: 899/1540
ucec: 894/1540
ucec: 892/1540
ucec: 900/1540
ucec: 898/1540
ucec: 903/1540
ucec: 902/

ucec: 1357/1540
ucec: 1363/1540
ucec: 1362/1540
ucec: 1365/1540
ucec: 1359/1540
ucec: 1364/1540
ucec: 1369/1540
ucec: 1354/1540
ucec: 1366/1540
ucec: 1370/1540
ucec: 1367/1540
ucec: 1368/1540
ucec: 1376/1540
ucec: 1371/1540
ucec: 1372/1540
ucec: 1373/1540
ucec: 1375/1540
ucec: 1378/1540
ucec: 1380/1540
ucec: 1379/1540
ucec: 1377/1540
ucec: 1383/1540
ucec: 1384/1540
ucec: 1381/1540
ucec: 1386/1540
ucec: 1385/1540
ucec: 1387/1540
ucec: 1390/1540
ucec: 1382/1540
ucec: 1389/1540
ucec: 1388/1540
ucec: 1393/1540
ucec: 1392/1540
ucec: 1391/1540
ucec: 1396/1540
ucec: 1394/1540
ucec: 1398/1540
ucec: 1397/1540
ucec: 1401/1540
ucec: 1395/1540
ucec: 1400/1540
ucec: 1403/1540
ucec: 1374/1540
ucec: 1399/1540
ucec: 1407/1540
ucec: 1402/1540
ucec: 1405/1540
ucec: 1406/1540
ucec: 1411/1540
ucec: 1409/1540
ucec: 1404/1540
ucec: 1413/1540
ucec: 1408/1540
ucec: 1410/1540
ucec: 1414/1540
ucec: 1417/1540
ucec: 1419/1540
ucec: 1416/1540
ucec: 1415/1540
ucec: 1412/1540
ucec: 1418/1540
ucec: 1423/1540
ucec: 14

### Download Tiles

Make adjustments to urls to allow tiles to be requested:

In [35]:
dz_pdf = pdf
dz_pdf['url'] = dz_pdf['url'].str.replace('FIF=', 'DeepZoom=')

/Users/aadi/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


The following is the code used to download tile data. Small adjustments were made to this code to allow multiple instances of python, and to allow simultaneous download on a Google Compute Engine VM -- modifications that allowed parallelization and significant speed-ups in data acquisition:

In [94]:
img_path = '/Users/aadi/Documents/tcga_imgs/tiles/'


def process_download_tile(url_idx, df_slice, category, save_dir, x, y):
    img_filename = save_dir + df_slice['name'].iloc[url_idx] + '_' + str(x) + '_' + str(y) + ".jpg"
    img_url = df_slice['url'].iloc[url_idx]
    img_url = img_url[:-17]
    img_url = img_url+"_files/15/"+str(x)+'_'+str(y)+'.jpg'
    #print(img_url)
    try:
        urllib.request.urlretrieve(img_url, img_filename)
    except:
        pass
    #print(category+': '+str(url_idx+1)+'/'+str(len(df_slice['url'])+1)+' -- '+str(x)+','+str(y))


In [96]:
for category in folder_names:
    category_dir = img_path+category
    if not os.path.exists(category_dir):
        os.makedirs(category_dir)
    df_slice = dz_pdf[dz_pdf['collection'].str.contains(category)]
    for url_idx in range(0, len(df_slice['url'])):
        print(category+': '+str(url_idx))
        save_dir = '/Users/aadi/Documents/tcga_imgs/tiles/'+category+'/'+df_slice['name'].iloc[url_idx]+'/'
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
#        for x in range(0, 100):
        joblib.Parallel(n_jobs=16)(joblib.delayed(process_download_tile)(url_idx, df_slice, category, save_dir, x, y) for y in range(0,30) for x in range(0,100))


brca: 1000
brca: 1001
brca: 1002
brca: 1003
brca: 1004
brca: 1005
brca: 1006
brca: 1007
brca: 1008
brca: 1009
brca: 1010
brca: 1011
brca: 1012
brca: 1013
brca: 1014
brca: 1015
brca: 1016
brca: 1017
brca: 1018
brca: 1019
brca: 1020
brca: 1021
brca: 1022
brca: 1023
brca: 1024
brca: 1025
brca: 1026
brca: 1027
brca: 1028
brca: 1029
brca: 1030
brca: 1031
brca: 1032
brca: 1033
brca: 1034
brca: 1035
brca: 1036
brca: 1037
brca: 1038
brca: 1039
brca: 1040
brca: 1041
brca: 1042
brca: 1043
brca: 1044
brca: 1045
brca: 1046
brca: 1047
brca: 1048
brca: 1049


KeyboardInterrupt: 